In [88]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
import pickle

In [89]:
df = pd.read_csv('/content/train.csv')
df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [90]:
df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1, inplace=True)
df.head()

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S

In [91]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

Test Train Split

In [92]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Survived',axis=1),df['Survived'], test_size=0.2, random_state=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [93]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         141
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [94]:
# Applying imputation
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

# Fit on X_train
si_age.fit(X_train[['Age']])
si_embarked.fit(X_train[['Embarked']])

# Transform X_train and X_test
X_train_age = si_age.transform(X_train[['Age']])
X_test_age = si_age.transform(X_test[['Age']])

X_train_embarked = si_embarked.transform(X_train[['Embarked']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

X_train_age.shape, X_test_age.shape, X_train_embarked.shape, X_test_embarked.shape

((712, 1), (179, 1), (712, 1), (179, 1))

In [95]:
# Applying OneHotEncoder

ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit on X_train
ohe_sex.fit(X_train[['Sex']])
ohe_embarked.fit(X_train_embarked)

# Transform X_train and X_test
X_train_sex = ohe_sex.transform(X_train[['Sex']])
X_test_sex = ohe_sex.transform(X_test[['Sex']])

X_train_embarked = ohe_embarked.transform(X_train_embarked)
X_test_embarked = ohe_embarked.transform(X_test_embarked)

X_train_sex.shape, X_test_sex.shape, X_train_embarked.shape, X_test_embarked.shape

((712, 2), (179, 2), (712, 3), (179, 3))

In [96]:
X_train_sex

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [97]:
X_train_embarked

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [98]:
# Concatenating

X_train_rem = X_train.drop(columns=['Sex','Age','Embarked'])
X_test_rem = X_test.drop(columns=['Sex','Age','Embarked'])

X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)

X_train_transformed.shape, X_test_transformed.shape

((712, 10), (179, 10))

In [99]:
# Applying Decision Tree

clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

In [100]:
y_pred = clf.predict(X_test_transformed)
y_pred

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1])

In [101]:
accuracy_score(y_test,y_pred)*100

77.6536312849162

In [103]:
# Saving Trained Model

pickle.dump(ohe_sex,open('Model_Training_Without_Pipelines/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('Model_Training_Without_Pipelines/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('Model_Training_Without_Pipelines/model.pkl','wb'))